In [1]:
#This is magic, don't touch it
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from html import unescape
import pandas as pd
import numpy as np
import re


In [13]:
with open('../corpus/registroactocom.csv', 'r') as file:
    unescaped_text = unescape(file.read())
    with open('../corpus/unescaped_text.csv', 'w') as unescaped_file:
        unescaped_file.write(unescaped_text)
    with open('../corpus/clean_text.csv', 'w') as clean_file:    
        clean_file.write(unescaped_text.replace('<p>', '')\
                         .replace('</p>', '')\
                         .replace('\r','')\
                         .replace('\xa0', ' '))
    with open('../corpus/regex_clean_text.csv','w') as regex_file:
        regex_file.write(re.sub('<.*?>','',unescaped_text, flags=re.DOTALL)\
                         .replace('\r','')\
                         .replace('\xa0', ' '))
    

In [14]:
raw_dataset = pd.read_csv('../corpus/regex_clean_text.csv', header=0)

In [15]:
raw_dataset.head()

,segmento_id,acto_id,titulo,palabrasclave,transcripcion,latitudp,longitudp
0,1,1,Presentación,NaN,DEMETRIA: Nada más que yo orita ando previnien...,19.449715,-101.731987
1,2,2,NaN,NaN,"MANUEL BAUTISTA: nosotros analizamos co, cómo,...",19.670720,-101.554051
2,3,5,Oficio de familia,Historia de vida,"00:00\nBERENICE: ¿Nos puede decir su nombre, s...",19.450582,-101.733160
3,4,4,Presentación,NaN,"VIDAL: […] Y, este… este año fue que , que hiz...",19.696462,-101.198387
4,499,22,Nuevo Amanecer,NaN,SANTIAGO: Y tampoco queremos este obligarlos a...,0.000000,0.000000


In [16]:
new_dataset = raw_dataset.copy()

In [17]:
new_dataset.drop(columns=['titulo','palabrasclave','acto_id','latitudp','longitudp'])

,segmento_id,transcripcion
0,1,DEMETRIA: Nada más que yo orita ando previnien...
1,2,"MANUEL BAUTISTA: nosotros analizamos co, cómo,..."
2,3,"00:00\nBERENICE: ¿Nos puede decir su nombre, s..."
3,4,"VIDAL: […] Y, este… este año fue que , que hiz..."
4,499,SANTIAGO: Y tampoco queremos este obligarlos a...
...,...,...
404,659,GEORGINA: ¿Y sí encontraron ellos el dinero?\n...
405,660,"GEORGINA: ¿O sea no a cualquiera les toca, pue..."
406,661,GEORGINA: ¿Su papá entonces tampoco agarro din...
407,662,GEORGINA: ¿Cómo hace cuanto dejo de escuchar q...


In [18]:
new_dataset.transcripcion[0]


'DEMETRIA: Nada más que yo orita ando previniendo para bañiar, [ah] eso es que lo andaba haciendo ya, sí.\nBERENICE: A ver, no nos, no nos tardamos mucho.\nDEMETRIA: Ajá, sí. \nBERENICE: ¿Quién va a preguntar?\nGEORGINA: Pues, si quiere empezamos con su nombre [mjm] ¿Cómo se llama usted?\nDEMETRIA: Yo me llamo Demetria.\nGEORGINA: Demetria.\nBERENICE: ¿Y cómo se apellida? ¿Demetria qué?\nDEMETRIA: Antonio Guzmán [mjm]. \nBERENICE: ¿Y cuántos?\nGEORGINA: ¿Y cuántos años tiene? \nDEMETRIA: No, pos ya tengo setenta, setenta y dos.\nITZI: ¿Y nació aquí en la Pacanda? \nDEMETRIA: Sí, aquí, somos de aquí pues, nativos, sí.\nGEORGINA: ¿Su esposo también?\nDEMETRIA: Sí, también él, también sí, también ya está grande pues también él. Él lleva más, ganó, este, 4 años, me gana a mí. [Oh] Sí, sí. \n '

In [19]:
new_dataset.to_csv('../corpus/new_dataset.csv')

In [20]:
corpus = new_dataset.transcripcion.astype('str')

In [21]:
tfidf_vectorizer = TfidfVectorizer(max_df= 0.5)

In [23]:
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
weights = np.asarray(tfidf_matrix.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tfidf_vectorizer.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(100)

,term,weight
1098,berenice,0.043811
6555,pus,0.042201
5245,mso,0.031414
8697,él,0.031383
5106,mi,0.031223
...,...,...
5715,pacanda,0.011601
3463,están,0.011578
8201,tú,0.011367
5823,parte,0.011363


http://www.ultravioletanalytics.com/blog/tf-idf-basics-with-pandas-scikit-learn

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html